In [15]:
import numpy as np
import pandas as pd
import joblib

In [16]:
def read_data(path, path_ihsg, 
              save_file = True,
              return_file = True,
              set_index = None):
    '''
    Read data from data folder in csv format.
    
    Parameters
    ----------
    path: str
          path to data
    
    '''
    
    emiten = pd.read_csv(path, index_col = set_index)
    ihsg = pd.read_csv(path_ihsg, index_col = set_index)
    
    def merge_emiten(emiten, ihsg):
        emiten = emiten.drop(['Open', 'High', 'Low', 'Adj Close'], axis = 1)
        ihsg = ihsg.drop(['Open', 'High', 'Low', 'Adj Close'], axis = 1)
        merged = pd.merge(emiten, ihsg, how='left', on='Date')
        merged.index = pd.to_datetime(merged.index)
        merged.rename(columns = {'Close_x':'Close', 'Volume_x':'Volume', 'Close_y':'Close_ihsg', 'Volume_y':'Volume_ihsg'}, inplace = True)
        merged['Close+1'] = merged['Close'].shift(-1)
        merged.dropna(inplace=True)
        merged.drop(merged.loc[merged["Volume"]==0].index, inplace=True)
        merged.drop(merged.loc[merged["Volume_ihsg"]==0].index, inplace=True)
        return merged

    if save_file:
        joblib.dump(merge_emiten(emiten, ihsg), "merged.pkl")
    
    if return_file:
        return merge_emiten(emiten, ihsg)



def split_input_output(dataset,
                       target_column,
                       save_file = True,
                       return_file = True):
    
    output_df = dataset[target_column]
    input_df = dataset.drop([target_column],
                            axis = 1)
    
    if save_file:
        joblib.dump(output_df, "output_df.pkl")
        joblib.dump(input_df, "input_df.pkl")
    
    if return_file:
        return output_df, input_df

def x_split(input_df, return_file=True, save_file=True):
    X_train = input_df[:int(input_df.shape[0]*0.6)]
    test_val = input_df[int(input_df.shape[0]*0.6):]
    X_val = test_val[:int(test_val.shape[0]*0.5)]
    X_test = test_val[int(test_val.shape[0]*0.5):]

    if save_file:
        joblib.dump(X_train, "X_train.pkl")
        joblib.dump(X_val, "X_val.pkl")
        joblib.dump(X_test, "X_test.pkl")

    if return_file:
        return X_train, X_val, X_test
        
def y_split(output_df, return_file=True, save_file=True):
    y_train = output_df[:int(output_df.shape[0]*0.6)]
    y_test_val = output_df[int(output_df.shape[0]*0.6):]
    y_val = y_test_val[:int(y_test_val.shape[0]*0.5)]
    y_test = y_test_val[int(y_test_val.shape[0]*0.5):]
    
    if save_file:
        joblib.dump(y_train, "y_train.pkl")
        joblib.dump(y_val, "y_valid.pkl")
        joblib.dump(y_test, "y_test.pkl")

    if return_file:
        return y_train, y_val, y_test

In [17]:
DATA_PATH = "data/AMRT.csv"
DATA_PATH_IHSG = "data/ihsg.csv"
TARGET_COLUMN = "Close+1"
INDEX_COLUMN = "Date"

data_house = read_data(DATA_PATH, DATA_PATH_IHSG,
                       set_index = INDEX_COLUMN)
output_df, input_df = split_input_output(
                            data_house,
                            TARGET_COLUMN)

X_train, X_val, X_test = x_split(input_df)
y_train, y_val, y_test = y_split(output_df)